In [14]:
import os
os.chdir("/dust3r")
print(os.getcwd())

/dust3r


In [15]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import json
import open3d as o3d

from dust3r.inference import inference_with_mask, create_gaussian_kernel
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from dust3r.cloud_opt.base_opt import global_alignment_loop
from masked_dust3r.scripts.utils.math import *
from masked_dust3r.scripts.utils.image import *


DATA_PATH = "/dust3r/masked_dust3r/data/jackal_training_data_0"
IMG_FILE_EXTENSION = ".png"
MASK_FILE_EXTENSION = ".png"

INIT_FRAMES = 50
NEW_FRAMES = 10
PREVIOUS_FRAMES = 40
TOTAL_FRAMES = 300

INIT_WEIGHT_FOCAL = 0.0001 * 0
INIT_WEIGHT_Z = 0.1 
INIT_WEIGHT_ROT = 0.01
INIT_WEIGHT_TRANS_SMOOTHNESS = 0.0001 
INIT_WEIGHT_ROT_SMOOTHNESS = 0.001 * 0

NEW_WEIGHT_FOCAL = 0.1 * 0
NEW_WEIGHT_Z = 0.1
NEW_WEIGHT_ROT = 0.1
NEW_WEIGHT_TRANS_SMOOTHNESS = 0.0001
NEW_WEIGHT_ROT_SMOOTHNESS = 0.00001

USE_COMMON_INTRINSICS = False

device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300 

In [16]:
GAUSSIAN_SIGMA = 51.0
SIZE = int(GAUSSIAN_SIGMA * 3)

kernel = create_gaussian_kernel(SIZE, GAUSSIAN_SIGMA).to(device)

SIZE = 11
kernel = torch.ones(SIZE, SIZE).to(device)

In [17]:
# Load the model

model_name = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth


instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>


In [18]:
images_array = []
masks_array = []

for i in range(30,60):
    images_array.append(os.path.join(DATA_PATH,"masked_images/{}{}".format(i,IMG_FILE_EXTENSION)))
    masks_array.append(os.path.join(DATA_PATH,"masks/{}{}".format(i,MASK_FILE_EXTENSION)))
images = load_images(images_array, size=512, verbose=True)
_,_,H,W = images[0]["img"].shape
masks = load_masks(masks_array, H, W, device)

>> Loading a list of 30 images
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/30.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/31.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/32.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/33.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/34.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/35.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/36.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/37.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_

In [19]:
pairs = make_pairs(images, scene_graph='swin-5', prefilter=None, symmetrize=True)
output = inference_with_mask(pairs, model, device, masks, kernel, batch_size=batch_size)
del pairs

>> Inference with model on 300 image pairs


100%|██████████| 300/300 [01:33<00:00,  3.20it/s]


In [20]:
#init_scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer)
#loss = init_scene.compute_global_alignment(init="mst", niter=niter, schedule='cosine', lr=lr)

#scene = init_scene

scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer, 
                        weight_focal = INIT_WEIGHT_FOCAL,
                        weight_z = INIT_WEIGHT_Z ,
                        weight_rot = INIT_WEIGHT_ROT  ,
                        weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS,
                        weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS)
#scene.im_poses = calculate_new_params(init_scene.im_poses,device)
#scene.im_focals = init_scene.im_focals
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

#averge_focal = scene.get_focals().sum().item()/len(images_array)
#fixed_focal = [averge_focal for _ in range(len(images_array))]
#mask = [True for _ in range(len(images_array))]
#scene.preset_focal(fixed_focal, mask)
#loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

#scene.weight_focal = INIT_WEIGHT_FOCAL
#scene.weight_z = INIT_WEIGHT_Z 
#scene.weight_rot = INIT_WEIGHT_ROT
#scene.weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS 
#scene.weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS
#loss = global_alignment_loop(scene, niter=niter, schedule=schedule, lr=lr)


 init edge (25*,28*) score=1.2694931030273438
 init edge (26*,28) score=1.254565715789795
 init edge (24*,28) score=1.2416644096374512
 init edge (26,21*) score=1.1983798742294312
 init edge (26,29*) score=1.2597357034683228
 init edge (23*,26) score=1.2589253187179565
 init edge (27*,29) score=1.2574924230575562
 init edge (23,22*) score=1.253562092781067
 init edge (27,0*) score=1.2523906230926514
 init edge (22,17*) score=1.2115325927734375
 init edge (27,1*) score=1.200950264930725
 init edge (22,18*) score=1.1934236288070679
 init edge (23,19*) score=1.1900578737258911
 init edge (0,3*) score=1.183078408241272
 init edge (2*,0) score=1.1731467247009277
 init edge (15*,18) score=1.1546553373336792
 init edge (0,4*) score=1.1542271375656128
 init edge (15,11*) score=1.1537435054779053
 init edge (23,20*) score=1.14914870262146
 init edge (0,5*) score=1.140466570854187
 init edge (1,6*) score=1.1186381578445435
 init edge (12*,15) score=1.2209951877593994
 init edge (13*,15) score=1.

100%|██████████| 300/300 [05:46<00:00,  1.16s/it, lr=1.27413e-06 loss=0.000182016]


In [21]:
all_focal = torch.stack(list(scene.im_focals))
all_poses = torch.stack(list(scene.im_poses))
Q = all_poses[:,:4]
Q = torch.nn.functional.normalize(Q, p=2, dim=1)
T = signed_expm1(all_poses[:,4:7])
tf = roma.RigidUnitQuat(Q, T).normalize()#.to_homogeneous()#.inverse()
tf_inv = tf.inverse()

off_z_axis = torch.tensor([]).to(device)

#print(tf.linear[:,3])

for e, (i, j) in enumerate(scene.edges):
    #print(torch.nn.functional.normalize((tf_inv[i] @ tf[j]).linear[:3],p=2, dim=0, eps=1e-12,))
    print(torch.nn.functional.normalize((tf[j] @ tf_inv[i]).linear[:3],p=2, dim=0, eps=1e-12)[2].abs())
    #off_z_axis = torch.cat((off_z_axis, 1-torch.nn.functional.normalize((tf[j] @ tf_inv[i]).linear[:3],p=2, dim=0, eps=1e-12)[2].abs().unsqueeze(0)))


tensor(0.9996, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9995, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9995, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(1.0000, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9990, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9987, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9999, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9994, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(1.0000, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9999, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9995, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9989, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9993, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9996, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9999, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(0.9982, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(1.0000, device='cuda:0', grad_fn=<AbsBackward0>)
tensor(1.0000, device='cuda:0', grad_fn=<AbsBack

In [22]:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()
intrinsics = scene.get_intrinsics()

In [23]:
print(focals)

tensor([[480.4880],
        [477.2055],
        [475.2227],
        [447.0557],
        [401.6037],
        [438.0927],
        [461.9557],
        [506.2375],
        [480.8504],
        [449.1308],
        [477.4696],
        [453.0122],
        [466.8129],
        [485.5738],
        [486.1373],
        [489.1867],
        [495.7773],
        [482.2085],
        [497.3250],
        [530.1573],
        [506.1595],
        [533.7809],
        [484.5402],
        [513.3201],
        [523.6434],
        [538.1131],
        [523.4783],
        [513.8554],
        [504.4046],
        [480.6968]], device='cuda:0', grad_fn=<ExpBackward0>)


In [24]:
#Check if pointclouds folder exists
#If exists, delete all files in the folder
if os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    for file in os.listdir("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
        os.remove("{DATA_PATH}/pointclouds/{file}".format(DATA_PATH=DATA_PATH, file=file))
        
if not os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    os.makedirs("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH))

for i in range(len(images)):
    pointcloud = pts3d[i].detach().cpu().numpy()
    pointcloud = pointcloud.reshape(-1, 3)
    color = imgs[i].reshape(-1, 3)
    confidence_mask = confidence_masks[i].detach().cpu().numpy()
    confidence_mask = confidence_mask.reshape(-1)
    
    masked_pointcloud = []
    masked_color = []

    for j in range(len(confidence_mask)):
        if confidence_mask[j]:
            masked_pointcloud.append(pointcloud[j])
            masked_color.append(color[j])

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(masked_pointcloud)
    pcd.colors = o3d.utility.Vector3dVector(masked_color)
    o3d.io.write_point_cloud("{DATA_PATH}/pointclouds/pointcloud{i}.ply".format(DATA_PATH=DATA_PATH, i=i), pcd)

In [25]:
#Create transform file

img = Image.open(images_array[0])
width, height = img.size
RESCALE_FACTOR  = width/512

transforms = {}
transforms["camera_model"] = "OPENCV"
if USE_COMMON_INTRINSICS:
    intrinsic_mean = intrinsics.mean(dim=0)
    transforms["fl_x"] = intrinsic_mean[0,0].item() * RESCALE_FACTOR
    transforms["fl_y"] = intrinsic_mean[1,1].item() * RESCALE_FACTOR
    transforms["w"] = width 
    transforms["h"] = height 
    transforms["cx"] = intrinsic_mean[0,2].item() * RESCALE_FACTOR
    transforms["cy"] = intrinsic_mean[1,2].item() * RESCALE_FACTOR

transforms["frames"] = []

OPENGL = np.array([[1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, -1, 0],
                    [0, 0, 0, 1]])

for i in range(len(poses)):
    if not((confidence_masks[i]==0).all()):
        frame = {}
        frame["file_path"] = "/".join(images_array[i].split("/")[-2:])
        #frame["transform_matrix"] = np.linalg.inv(poses[i].detach().cpu().numpy()).tolist()
        frame["transform_matrix"] = np.dot(poses[i].detach().cpu().numpy(),OPENGL).tolist()
        frame["mask_path"] = "/".join(masks_array[i].split("/")[-2:])
        transforms["frames"].append(frame)
        
        if not USE_COMMON_INTRINSICS:
            frame["fl_x"] = intrinsics[i,0,0].item() * RESCALE_FACTOR
            frame["fl_y"] = intrinsics[i,1,1].item() * RESCALE_FACTOR
            frame["cx"] = intrinsics[i,0,2].item() * RESCALE_FACTOR
            frame["cy"] = intrinsics[i,1,2].item() * RESCALE_FACTOR
            img = Image.open(images_array[i])
            width, height = img.size
            transforms["w"] = width 
            transforms["h"] = height 

#Save transform file
with open("{}/transforms.json".format(DATA_PATH), 'w') as f:
    json.dump(transforms, f, indent=4)

In [26]:
import roma

all_poses = torch.stack(list(scene.im_poses))
Q = all_poses[:,:4]
Q = torch.nn.functional.normalize(Q, p=2, dim=1)
T = signed_expm1(all_poses[:,4:7])
tf = roma.RigidUnitQuat(Q, T).normalize().to_homogeneous()

OPENGL = torch.tensor([[1, 0, 0, 0],
                       [0, -1, 0, 0],
                       [0, 0, -1, 0],
                       [0, 0, 0, 1]], dtype=torch.float32).to(device)

tf = torch.matmul(tf, OPENGL)

tf = roma.RigidUnitQuat(Q, T).normalize()
print(tf.linear[:,0]/tf.linear[:,3])

tensor([ 0.0831,  0.0851,  0.0876,  0.0898,  0.0920,  0.0938,  0.0953, -0.1569,
        -0.0282, -0.0094,  0.0036,  0.0133,  0.0219,  0.0295,  0.0361,  0.0420,
         0.0476,  0.0524,  0.0575,  0.0609,  0.0712,  0.0741,  0.0764,  0.0801,
         0.0827,  0.0863,  0.0866,  0.0877,  0.0896,  0.0918], device='cuda:0',
       grad_fn=<DivBackward0>)
